In [1]:
# dependencies
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
from citipy import citipy
import numpy as np
import time
import os

# establish openweather API key/url
file_name = "../../../PythonScripts/BootcampExercises/API_Keys/api_keys.json"
data = json.load(open(file_name))
key = data['openweather']
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url = url + "appid=" + key + "&units=" + units + "&q="

# get olympic data files
csv_path = os.path.join("../Resources", "GDP_allyears.csv") #do not make changes
gdp = pd.read_csv(csv_path)
csv_path = os.path.join("../Resources", "country_participation.csv") #do not make changes
participation = pd.read_csv(csv_path)
csv_path = os.path.join("../Resources", "country_codes.csv") #do not make changes
codes = pd.read_csv(csv_path)

In [2]:
# clean/format participation data set
participation["TotalMedals"] = participation["Gold"] + participation["Silver"] + participation["Bronze"]
participation = participation.fillna(0)
participation["From"] = participation["From"].astype(int)
participation["To"] = participation["To"].astype(int)
participation["Gold"] = participation["Gold"].astype(int)
participation["Silver"] = participation["Silver"].astype(int)
participation["Bronze"] = participation["Bronze"].astype(int)
participation["Sports"] = participation["Sports"].astype(int)
participation["Participants"] = participation["Participants"].astype(int)
participation["TotalMedals"] = participation["TotalMedals"].astype(int)

In [8]:
# determine number of participation years
gameyears = [1924, 1928, 1932, 1936, 1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1994, 1998, 2002, 2006, 2010, 2014]
for index, row in participation.iterrows():
    counter = 0
    To = row[2]
    From = row[1]
    for year in gameyears:
        if year >= From and year <= To:
            counter = counter+1
    participation.loc[index, "TotalYears"] = counter
participation["TotalYears"] = participation["TotalYears"].astype(int)

In [11]:
# find average medals per year per country
participation["AvgMedals"] = participation["TotalMedals"]/participation["TotalYears"]
participation["AvgMedals"] = round(participation["AvgMedals"])

In [12]:
participation

,Country,From,To,Participants,Sports,Gold,Silver,Bronze,TotalMedals,TotalYears,AvgMedals
0,Australia,1924,2014,240,14,5,3,4,12,22,0.545455
1,Austria,1924,2014,859,14,59,78,81,218,22,9.909091
2,Belgium,1924,2014,124,8,1,1,3,5,22,0.227273
3,Canada,1924,2014,1279,15,62,56,52,170,22,7.727273
4,Czechoslovakia,1924,1992,480,11,2,8,15,25,16,1.562500
5,Finland,1924,2014,723,12,42,62,57,161,22,7.318182
6,France,1924,2014,823,16,31,31,47,109,22,4.954545
7,Great Britain,1924,2014,642,16,10,4,12,26,22,1.181818
8,Hungary,1924,2014,206,10,0,2,4,0,22,0.000000
9,India,1924,2014,21,4,0,0,0,0,22,0.000000
